In [258]:
import pandas as pd
import numpy as np

In [259]:
data_dir = 'data/'
filename = 'city_temperature.csv'

In [260]:
df = pd.read_csv(data_dir + filename)
df.head()

/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_33502/434626652.py:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_dir + filename)


,Region,Country,State,City,Month,Day,Year,AvgTemperature
0,Africa,Algeria,NaN,Algiers,1,1,1995,64.2
1,Africa,Algeria,NaN,Algiers,1,2,1995,49.4
2,Africa,Algeria,NaN,Algiers,1,3,1995,48.8
3,Africa,Algeria,NaN,Algiers,1,4,1995,46.4
4,Africa,Algeria,NaN,Algiers,1,5,1995,47.9


In [261]:
df['Region'].value_counts()/len(df)
yes_label = 'North America'
df['Label'] = df.apply(lambda x: 1 if x['Region'] == yes_label else 0, axis=1)

In [262]:
df['Label'].value_counts()/len(df)

1    0.535618
0    0.464382
Name: Label, dtype: float64

In [263]:
df[df['Year'].isin(range(2000, 2005))]['Country'].value_counts()


US                   292241
Canada                18270
Australia              9135
China                  9135
India                  7308
                      ...  
Myanmar (Burma)        1827
Mongolia               1827
Malaysia               1827
Laos                   1827
Serbia-Montenegro      1601
Name: Country, Length: 125, dtype: int64

In [264]:
# for each unique country, state, city, turn the average temperatures over the days into a time series
df_2000 = df[df['Year'].isin(range(2000, 2005))]
df_2000['State'].fillna('NA', inplace=True)
df_2000 = df_2000.groupby(['Year', 'Region', 'Country', 'State', 'City'])['AvgTemperature'].apply(list)
df_2000 = df_2000.reset_index()
df_2000['Label'] = df_2000.apply(lambda x: 1 if x['Region'] == yes_label else 0, axis=1)
print(df_2000['Label'].value_counts()/len(df_2000))
df_2000.head()
df_2000['len_ts'] = df_2000['AvgTemperature'].apply(lambda x: len(x))
df_2000['len_ts'].value_counts()

1    0.526802
0    0.473198
Name: Label, dtype: float64


/var/folders/_6/_8xn00vj0gz5g10qr8khtxz40000gn/T/ipykernel_33502/3211531057.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2000['State'].fillna('NA', inplace=True)


365    967
366    642
730      6
732      4
211      1
304      1
140      1
287      1
Name: len_ts, dtype: int64

In [265]:
def day_ts_to_month_ts(ts):
    months = [0, 31, 31+28, 31+28+31, 31+28+31+30, 31+28+31+30+31, 31+28+31+30+31+30, 31+28+31+30+31+30+31, 31+28+31+30+31+30+31+31, 31+28+31+30+31+30+31+31+30, 31+28+31+30+31+30+31+31+30+31, 31+28+31+30+31+30+31+31+30+31+30, 31+28+31+30+31+30+31+31+30+31+30+31]
    if len(ts) == 366: # leap year
        for i in range(2, 13):
            months[i] += 1
    elif len(ts) != 365:
        return None
    month_ts = []
    for i in range(1, 13):
        month_ts.append(round(np.mean(ts[months[i-1]:months[i]]), 1))
    return month_ts


In [266]:
df_2000['month_ts'] = df_2000['AvgTemperature'].apply(day_ts_to_month_ts)

In [267]:
df_2000.dropna(inplace=True)

In [268]:
df_2000[['temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'temp6', 'temp7', 'temp8', 'temp9', 'temp10', 'temp11', 'temp12']] = pd.DataFrame(df_2000['month_ts'].tolist(), index=df_2000.index)

In [269]:
df_2000['month_ts'].iloc[0]

[47.2, 52.1, 56.4, 61.2, 68.1, 72.4, 78.1, 79.5, 74.1, 65.5, 59.6, 55.5]

ML Benchmark

In [270]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

X_features = ['temp1', 'temp2', 'temp3', 'temp4', 'temp5', 'temp6', 'temp7', 'temp8', 'temp9', 'temp10', 'temp11', 'temp12']
y_label = 'Label'

X_train, X_test, y_train, y_test = train_test_split(df_2000[X_features], df_2000[y_label], test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train, y_train)
print('accuracy: ', accuracy_score(y_test, model.predict(X_test)))
print('roc_auc: ', roc_auc_score(y_test, model.predict_proba(X_test)[:, 1]))


accuracy:  0.7484472049689441
roc_auc:  0.769720592775548


/Users/kylecox/opt/anaconda3/envs/ml/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [276]:
# to dictionary
data_dict = df_2000[['Year', 'Region', 'Country', 'State', 'City', 'month_ts', 'Label']].rename(columns={'month_ts': 'MonthlyAvgTemperature'}).to_dict(orient='index')


In [278]:
import pickle
with open('data/data_dict.pkl', 'wb') as handle:
    pickle.dump(data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)